# Complete Dataframe with the full Radioteca

This notebook is the second part of the creation of the dataframe containing all the data from the different corpora adquired. The separate corpora were turned into a dataframes and explored in [Data-Parsing-Exploratory-Analysis-Part1](https://github.com/Data-Science-for-Linguists-2025/Linguistic-Markers-Catalan-Substitution/blob/main/Data-Parsing-Exploratory-Analysis-Part1.ipynb.) individually. In this notebook they are merged following the same steps as in our first approach without the full radioteca except when dealing with radioteca.

In [1]:
# importing libraries and tools
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

Some of the data frames come with a great amount of metadata that won't be necessary for our research purposes. To keep it we will maintain the separate dataframes and create a merged one extracting only the columns necessary for our research question. That will be the title, date and text/content. We will also create other columns that will be useful for our Exploratory Data Analysis. 

In [2]:
# loading the individual dataframes from their pickled form
CTILC_df = pd.read_pickle("Data/CTILC.pkl")
parlament_parla_df = pd.read_pickle("Data/parlaparla.pkl")
parlaMint_df = pd.read_pickle("Data/parlaMint.pkl")

**Comment:** Only the first three dataframes CTILC, Parlament Parla and Parlamint have been loaded as we will treat radioteca differently because of its huge dimensions. We will load the data in chuncks instead of all at once as otherwise it would make the notebook crash. Moreover, as this notebook is the second take of the merging of the dataframes, as previously we had only scraped part of radioteca, and process was re-done after scraping the enirety of radioteca df, we already know it will give us a very imbalenced distribution of the data. It is a massive dataframe but it will only bring in lots of data about the same years, which is not what we are looking for. Therefore, we will perform 2 strategies to reduce the amount of data and create a sample of the whole radioteca dataframe while trying to avoid any biases. 

The strategies, as in the original notebook [Data-Parsing-Exploratory-Analysis-2](https://github.com/Data-Science-for-Linguists-2025/Linguistic-Markers-Catalan-Substitution/blob/main/Data-Parsing-Exploratory-Analysis-2.ipynb), will be:\
**Strategy 1:Limiting Individual Contribution**\
Aiming for the most unbiased and representative data, we will first drop individual Speaker's contributions if they are over 1500 characters (about 200 words) long for the Radioteca data. This will allow for a less speaker-specific analysis.\
**Strategy 2: Balancing Program  Contribution**\
Since Radioteca has a lot of metadata, we will also use another piece of data ensure a more representative distribution while reducing the data size by limiting a show's episode contribution to 1500 characters as well. That way, we are also ensuring a more diverse corpora topic-wise and less show-specific data.

We will do it without loading the full dataset completely, year file per year file.

In [3]:
# function for strategy 1, speaker limit (we will reuse it for the other larger dataframes
def under1500(dataframe, contrivutorColName):
    '''
    takes in a dataframe
    after the cumulative text length of one speaker goes over 1500 characters
    the following contributions are no longer added to the dataframe
    ensuring a max of 1500 characters per speaker/contributor
    prints out the maximum and minimum length per contributor before and after the change
    '''
    if contrivutorColName == "Speaker":
        dataframe["CUMSUM_len"] = dataframe.groupby(["Episode", contrivutorColName])["Text_len"].cumsum()
    else:
        dataframe["CUMSUM_len"] = dataframe.groupby(contrivutorColName)["Text_len"].cumsum()
    max1500_df = dataframe[dataframe["CUMSUM_len"] <= 1500]
    if contrivutorColName == "Speaker":
        before_spkcount = dataframe.groupby(["Episode", contrivutorColName])["Text_len"].sum()
        after_spkcount = max1500_df.groupby(["Episode", contrivutorColName])["Text_len"].sum()
    else:
        before_spkcount = dataframe.groupby(contrivutorColName)["Text_len"].sum()
        after_spkcount = max1500_df.groupby(contrivutorColName)["Text_len"].sum()

    #printing contribution max and min to ensure the process worked out well
    # print("before:", before_spkcount.max(), "-", before_spkcount.min(), "after:", after_spkcount.max(), "-", after_spkcount.min())
    return max1500_df

**Comment:** In the following code we are loading the the year files, as the dataframe was saved in several parquet files per year.\
We are creating a list with all of the radioteca_year.parquet files. Then, for each of the files, we are looking for the year in the file name so we can use it when we save it. For the data-reducing strategies, then, we first use the previously defined under1500 function to limit speaker contributions to 1500 tokens per speaker. Then we grouping the texts per episode, and when an episode's contributions goes over 1500 characters it is no longer added to the dataframe. Lastly we are turning the reduced dataframe, after applying the two strategies into a new parquet file in our data folder. We are also printing out the changes in the number of character maximums per year file, before and after the reduction.

In [4]:
import glob as glob
from tqdm import tqdm 

results = []
files_per_year = glob.glob("data/radioteca_*.parquet")

for f in files_per_year:
    year_pattern = re.compile(r"\d+")
    year = re.search(year_pattern, f).group()
    radioteca_Ydf = pd.read_parquet(f)

    print(f"radioteca_year_{year}_reduced.parquet")
    
    # Strategy 1: limit speaker contributions to 1500 characters
    radioteca_Ydf = under1500(radioteca_Ydf, "Speaker")

    # Strategy 2: limit episode contributions to 1500 characters
    radioteca_Ydf["CUMSUM_ep"] = radioteca_Ydf.groupby("Episode")["Text_len"].cumsum()
    radioteca_Ydf = radioteca_Ydf[radioteca_Ydf["CUMSUM_ep"] <= 1500]

    radioteca_Ydf.to_parquet(f"data/radioteca_year_{year}_reduced.parquet")

radioteca_year_2004_reduced.parquet
radioteca_year_1986_reduced.parquet
radioteca_year_2003_reduced.parquet
radioteca_year_2020_reduced.parquet
radioteca_year_2013_reduced.parquet
radioteca_year_2025_reduced.parquet
radioteca_year_2012_reduced.parquet
radioteca_year_1992_reduced.parquet
radioteca_year_1995_reduced.parquet
radioteca_year_2024_reduced.parquet
radioteca_year_2015_reduced.parquet
radioteca_year_2003_reduced.parquet
radioteca_year_1987_reduced.parquet
radioteca_year_2012_reduced.parquet
radioteca_year_2002_reduced.parquet
radioteca_year_1994_reduced.parquet
radioteca_year_2014_reduced.parquet
radioteca_year_2019_reduced.parquet
radioteca_year_2009_reduced.parquet
radioteca_year_2002_reduced.parquet
radioteca_year_2000_reduced.parquet
radioteca_year_2010_reduced.parquet
radioteca_year_1995_reduced.parquet
radioteca_year_2005_reduced.parquet
radioteca_year_2011_reduced.parquet
radioteca_year_2001_reduced.parquet
radioteca_year_1994_reduced.parquet
radioteca_year_2021_reduced.

In [5]:
# getting all the reduced files paths
reduced_files = glob.glob("data/radioteca_*_reduced.parquet")

In [6]:
# printing the first one to take a look
print(reduced_files[0])
print()

data/radioteca_year_2004_reduced.parquet



Next we will concatenate all of the reduced dataframes from the parquets by appending them to a list and then concatenating them to a newly creadted radioteca_reduced dataframe.

In [7]:
radioteca_reduced = pd.DataFrame() # creating the dataframe

data_per_year = []

for file in tqdm(reduced_files):
    year_df = pd.read_parquet(file)
    data_per_year.append(year_df)

df = pd.concat(data_per_year, ignore_index=True)
del data_per_year 

100%|███████████████████████████████████████████| 36/36 [00:01<00:00, 19.70it/s]


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1648634 entries, 0 to 1648633
Data columns (total 13 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   Line_id     1648634 non-null  object        
 1   Speaker     1648634 non-null  object        
 2   Time        1648634 non-null  object        
 3   Text        1648634 non-null  object        
 4   Date        1648634 non-null  datetime64[ns]
 5   Station     1641081 non-null  object        
 6   Show        1648634 non-null  object        
 7   Episode     1648634 non-null  object        
 8   URL         1648634 non-null  object        
 9   Year        1648634 non-null  int32         
 10  Text_len    1648634 non-null  int64         
 11  CUMSUM_len  1648634 non-null  int64         
 12  CUMSUM_ep   1648634 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(3), object(8)
memory usage: 157.2+ MB


**Comment:** The data is much smaller and a bit more manageable now. It dropped from GBs of memory usage to a 157MB.

As mentioned previously we don't need a lot of the metadata we are storing in the dataframes, therefore we will just keep the columns we need. That is Year, ID, Text and Text_length.

In [9]:
# creating a reduced dataframe with only the date info, title of the file, content/text columns and its length
reduced_CTILC = CTILC_df.filter(["Year", "Title", "Text", "Text_len"])
reduced_parlament_parla = parlament_parla_df.filter(["Path", "Sentence", "Text", "Text_len"])
reduced_parlamint = parlaMint_df.filter(["Date", "Title", "Text", "Text_len"])
reduced_radioteca = df.filter(["Year", "Line_id", "Text", "Text_len"])

As we will be next combining all dataframes we will create a source column to avoid loosing the data of where the row came from after merging.

In [10]:
# adding a Source Corpora column to keep the data
reduced_CTILC["Source_corpora"] = "CTILC"
reduced_parlament_parla["Source_corpora"] = "Parlament Parla"
reduced_parlamint["Source_corpora"] = "ParlaMint"
reduced_radioteca["Source_corpora"] = "Radioteca.cat"

We will only keep the year infomation as the date in the corpora parlamint and the radioteca data frame where the full (month, day, year) information was provided.

In [11]:
# casting year as integer
print(type(reduced_radioteca["Year"][0])) # it is currently a float that we extracted from the complete date
reduced_radioteca["Year"] = reduced_radioteca["Year"].apply(int)
print(type(reduced_radioteca["Year"][0])) # checking if data type is now what we need

<class 'numpy.int32'>
<class 'numpy.int64'>


There is no date metadata provided for the parlament parla dataframe.\
For our purposes, as we need a date we will make an aproximation based off the data given by the owners.\
Initially, on the original notebook, as Parlament Parla is presented as a corpora containing data from 2007 to 2015 I assigned the date aproximation 2010 to all the data in the Parlament Parla dataframe. However, that made the data cluster all on the same year which resulted in an even bigger imbalance in our data. Therefore, we will go straight into the second approach I took to date this dataframe. That is, distributing the parlament-parla files evenly throughout the years it covers (2007-2018) instead of approximating all of them as 2015.\

In [12]:
reduced_parlament_parla.head(1)

,Path,Text,Text_len,Source_corpora
0,clean_train/3/1/31ca4d158eaef166c37a_18.87_23....,perquè que el president de catalunya sigui reb...,85,Parlament Parla


In [13]:
# stablishing number of rows/documents/contributions that parlament parla contains 
size = reduced_parlament_parla.shape[0]
print(size)

87356


In [14]:
# creating a numpy array containing 3849 random years between 2007 to 2018, both included
import numpy as np
years = np.random.randint(2007, 2019, size) # 2019 will be excluded

In [15]:
# turning the numpy array into a list
reduced_parlament_parla["Year"] = years.tolist()

In [16]:
# checking the documents per year distribution we ended up with after the random year assignantion 
print(reduced_parlament_parla["Year"].value_counts())
print()
print("Total:")
print(reduced_parlament_parla["Year"].value_counts().sum())
print()
print("Mean:")
print(reduced_parlament_parla["Year"].value_counts().mean())
print()
print("Standard deviation:")
print(reduced_parlament_parla["Year"].value_counts().std())
print()

Year
2011    7488
2017    7357
2014    7340
2009    7321
2008    7298
2012    7293
2007    7286
2015    7222
2013    7212
2010    7188
2018    7178
2016    7173
Name: count, dtype: int64

Total:
87356

Mean:
7279.666666666667

Standard deviation:
92.10007732040317



**Comment:**\
We ended up with a decently even spread through the years. All the years are represented with around 320 pieces of data.
A standard deviation of 15 contributions of data, considering that we are working with a total of 3849, seems to be a reasonably uniform distribution that we can use, being mindful that it is not the real date of the documents and just an approximation. We should keep the data from 2007 to 2018 as grouped as possible to avoid making misanalyses or drawing incorrect conclusions.

In [17]:
# adjusting column names to match before concatenating
reduced_CTILC = reduced_CTILC.rename(columns ={"Year":"Year", "Title":"Line_id", "Text":"Text", 
                                                                   "Source_corpora":"Source_corpora", "Text_len":"Text_len"})
reduced_parlament_parla = reduced_parlament_parla.rename(columns ={"Year":"Year", "Path":"Line_id", "Text":"Text", 
                                                                   "Source_corpora":"Source_corpora", "Text_len":"Text_len"})
reduced_parlamint = reduced_parlamint.rename(columns ={"Date":"Year", "Title":"Line_id", "Text":"Text", 
                                                       "Source_corpora":"Source_corpora", "Text_len":"Text_len"})

In [18]:
# concatenating all datasets' relevant columns in a single data frame
complete_data = pd.concat([reduced_CTILC, reduced_parlament_parla, reduced_parlamint, reduced_radioteca]) 

In [19]:
complete_data.head()

,Year,Line_id,Text,Text_len,Source_corpora
0,1926,Discurs llegit per... donar a conèxer la perso...,"L'home que per amor al estudi, impulsat per un...",37497,CTILC
1,1920,Parlament llegit en la festa inaugural de l'Or...,"Cantaires de la Garriga, Senyores i senyors:\n...",9253,CTILC
2,1900,Discurs-pròlec,Discurs-prolec Llegit en la societat mèdic-far...,73881,CTILC
3,1894,Discurs,"Senyors excelentissims, senyors:\n\nQuan rebí ...",29393,CTILC
4,1903,Discurs,"Senyors:\n\nSembla que era air, y fa ja uns qu...",26577,CTILC


In [20]:
print(complete_data.isna().sum())

Year              0
Line_id           0
Text              0
Text_len          0
Source_corpora    0
dtype: int64


In [21]:
# sorting joined data by year, from oldest to most recent
complete_data["Year"] = complete_data["Year"].apply(lambda x : int(x)) # casting all Years as integers, as we have some strings mixed up
complete_data.sort_values(["Year"])

ValueError: invalid literal for int() with base 10: '2015-10-26'

In [ ]:
# resetting index after sorting
complete_data = complete_data.reset_index()

In [ ]:
# pickling the complete joined data frame
complete_data.to_pickle("myfulldata.pkl")

In [ ]:
#creating a dictionary of the cumulative amount of characters per year
year_length = {}
# setting year as the index of the dataframe
complete_data = complete_data.set_index("Year")
# iterating over the rows 
for year, row in complete_data.iterrows():
    if year not in year_length:
        year_length[year] = row["Text_len"]  # initializing year's length total
    else:
        year_length[year] += row["Text_len"]  # accumulating length to already present year

In [ ]:
year_length = dict(sorted(year_length.items(), key=lambda item: item[1]))
for key in year_length.keys():
    value = year_length[key]
    #print(key, "-", value)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  

# placeholder y-values for timeline
years = list(year_length.keys())  
y_values = np.ones(len(years)) 

# creating mosaic layout for our multiple plots
fig, ax = plt.subplot_mosaic([["B", "B"],  # Timeline
                              ["A", "A"],  # Histogram
                              ["D", "D"]],  # Text count & corpus length
                             figsize=(10, 8),
                             constrained_layout=True)

# (A) Histogram of Document Distribution Over Time
ax["A"].hist(complete_data.index, bins=min(20, len(years)), histtype="step", color="blue", lw=1.5)
ax["A"].ticklabel_format(style='plain', axis='y')
ax["A"].set_title("Document Distribution Over Time")
ax["A"].set_xlabel("Year")
ax["A"].set_ylabel("Count")

# (B) Timeline (Scatter Plot)
ax["B"].scatter(years, y_values, color="blue", marker="o", lw=1.5)
ax["B"].set_title("Timeline of Documents")
ax["B"].set_xlabel("Year")
ax["B"].set_yticks([])  # Remove y-axis labels since they are not meaningful
ax["B"].grid(axis="x")

# (D) Text length per year in character counts
capped_values = [min(val, 10_000_000) for val in year_length.values()]  # limit to 10M

ax["D"].barh(list(year_length.keys()), capped_values, color="blue")
ax["D"].set_title("Text Length per Year (capped after 10M)")  
ax["D"].set_xlabel("Text Length")  

# Fix Tick Labels (Without `ticker`)
x_ticks = ax["D"].get_xticks()  
ax["D"].set_xticks(x_ticks)  
ax["D"].set_xticklabels([f"{int(x):,}" for x in x_ticks])  
ax["D"].set_xlim(0, 10_000_000)

# displaying the plots' mosaic
plt.show()